# 문제 풀이 프로세스

1. 데이터 유형 파악
df.info() => 메모장에 복붙
2. 전처리
  - X, y, train/test set으로 분리
  - 분리된 shape 확인
  - 결측치 처리
  - 수치형 변수 스케일링(min_max, standard)
  - 범주형 변수 인코딩
3. 데이터 분리
4. 모델 학습 및 검증
5. 평가
6. 결과 저장


In [1]:
import pandas as pd

train = pd.read_csv("../data/customer_train.csv")
test = pd.read_csv("../data/customer_test.csv")

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   회원ID     3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   방문일수     3500 non-null   int64  
 7   방문당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2482 entries, 0 to 2481
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   회원ID     2482 non-null   int64  
 1   총구매액     2482 non-null   int64  
 2   최대구매액    2482 non-null   int64  
 3   환불금액     871 non-null    float64
 4   주구매상품    2482 non-null   object 
 5   주구매지점    2482 non-null   object 
 6   방문일수     2482 non-null   int64  
 7   방문당구매건수  2482 non-null   float64
 8   주말방문비율   2482 non-null   float64
 9   구매주기     2482 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 194.0+ KB


In [4]:
import pandas as pd

train = pd.read_csv("../data/customer_train.csv")
test = pd.read_csv("../data/customer_test.csv")

X_train = train.drop(["총구매액"], axis=1)
y_train = train["총구매액"]

X_test = test.drop(["총구매액"], axis=1)
# print(X_train.shape, y_train.shape, X_test.shape)

# print(X_train.isna().sum())
X_train["환불금액"] = X_train["환불금액"].fillna(0)
X_test["환불금액"] = X_test["환불금액"].fillna(0)

X_train = X_train.drop(["회원ID"], axis=1)
X_test = X_test.drop(["회원ID"], axis=1)

from sklearn.preprocessing import StandardScaler, LabelEncoder
scaler = StandardScaler()
encoder = LabelEncoder()
n_cols = X_train.select_dtypes(exclude="object").columns
s_cols = X_train.select_dtypes(include="object").columns
X_train[n_cols] = scaler.fit_transform(X_train[n_cols])
X_test[n_cols] = scaler.transform(X_test[n_cols])

for col in s_cols:
    X_train[col] = encoder.fit_transform(X_train[col])
    X_test[col] = encoder.transform(X_test[col])

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
model = RandomForestRegressor()
model1 = GradientBoostingRegressor()
model.fit(X_train, y_train)
model1.fit(X_train, y_train)
y_val_pred = model.predict(X_val)
y_val_pred1 = model1.predict(X_val)

from sklearn.metrics import root_mean_squared_error, accuracy_score
rmse = root_mean_squared_error(y_val, y_val_pred)
rmse1 = root_mean_squared_error(y_val, y_val_pred1)
# acc = accuracy_score(y_val, y_val_pred)
print(rmse, rmse1)

y_pred = model.predict(X_test)
result = pd.DataFrame(y_pred, columns=["pred"])
# result.to_csv("result.csv")
# 
# result = pd.read_csv("result.csv")
print(result.describe())

66709409.06944499 73921775.98312347
               pred
count  2.482000e+03
mean   9.996649e+07
std    1.459127e+08
min   -2.030760e+06
25%    6.023716e+06
50%    3.647512e+07
75%    1.438385e+08
max    1.297926e+09
